In [1]:
import aiohttp
import asyncio
import pandas as pd
from datetime import datetime, timedelta
from my_funnel_utils import load_api_tokens

In [2]:


async def get_funnel_v3(days_num: int, account: str, api_token: str):
    """Получение статистики по воронке продаж Wildberries"""
    products_list = []
    headers = {"Authorization": api_token}
    normal_delay = 2
    retry_delay = 20
    url = "https://seller-analytics-api.wildberries.ru/api/analytics/v3/sales-funnel/products"
    start = end = datetime.now() - timedelta(days=days_num)

    limit = 50
    offset = 0
    max_attempts = 10
    attempt = 0

    async with aiohttp.ClientSession(headers=headers) as session:
        while True:
            payload = {
                "selectedPeriod": {
                    "start": start.strftime("%Y-%m-%d"),
                    "end": end.strftime("%Y-%m-%d")
                },
                "limit": limit,
                "offset": offset
            }

            try:
                async with session.post(url, json=payload) as res:
                    if res.status == 200:
                        data = await res.json()
                        products = data.get("data", {}).get("products", [])

                        if not products:
                            print(f"📭 Нет данных для {account}")
                            break

                        for p in products:
                            p["account"] = account
                        products_list.extend(products)

                        print(f"✅ Получено {len(products_list)} товаров ({len(products)} новых) для {account}")

                        if len(products) < limit:
                            break

                        offset += len(products)
                        attempt = 0
                        await asyncio.sleep(normal_delay)

                    elif res.status == 429:
                        print(f"⚠️ Ошибка 429 для {account}: слишком много запросов, ждем {retry_delay} сек.")
                        await asyncio.sleep(retry_delay)
                        attempt += 1
                        if attempt >= max_attempts:
                            print(f"🚫 Превышено число попыток ({max_attempts}) для {account}")
                            break
                        continue

                    elif res.status in (400, 401, 403):
                        err = await res.json()
                        print(f"⚠️ Ошибка {res.status} для {account}: {err.get('message', 'Ошибка доступа')}")
                        return None

                    else:
                        print(f"⚠️ Неожиданный статус {res.status} для {account}")
                        attempt += 1
                        if attempt >= max_attempts:
                            break

            except aiohttp.ClientError as err:
                print(f"🌐 Сетевая ошибка: {err}")
                attempt += 1
                if attempt >= max_attempts:
                    break

            except Exception as e:
                print(f"💥 Неожиданная ошибка: {e}")
                break

    if products_list:
        print(f"🟢 Завершено получение данных по {account}. Всего товаров: {len(products_list)}")
        return products_list
    else:
        print(f"❌ Не удалось получить данные по воронке продаж для {account}")
        return None

    
async def main():
    # Создаем задачник для получения данных о поставках по всем аккаунтам асинхронно
    tasks = [get_funnel_v3(1, account, api_token) for account, api_token in load_api_tokens().items()]
    res = await asyncio.gather(*tasks)
    return res

In [3]:
result = await main()

✅ Получено 50 товаров (50 новых) для Оганесян
✅ Получено 50 товаров (50 новых) для Тоноян
✅ Получено 50 товаров (50 новых) для Старт
✅ Получено 50 товаров (50 новых) для Вектор
✅ Получено 50 товаров (50 новых) для Хачатрян
✅ Получено 50 товаров (50 новых) для Даниелян
✅ Получено 50 товаров (50 новых) для Старт2
✅ Получено 50 товаров (50 новых) для Лопатина
✅ Получено 50 товаров (50 новых) для Пилосян
✅ Получено 100 товаров (50 новых) для Оганесян
✅ Получено 100 товаров (50 новых) для Хачатрян
✅ Получено 100 товаров (50 новых) для Вектор
✅ Получено 100 товаров (50 новых) для Старт
✅ Получено 100 товаров (50 новых) для Даниелян
✅ Получено 100 товаров (50 новых) для Старт2
✅ Получено 100 товаров (50 новых) для Тоноян
✅ Получено 100 товаров (50 новых) для Лопатина
✅ Получено 100 товаров (50 новых) для Пилосян
✅ Получено 150 товаров (50 новых) для Вектор
✅ Получено 150 товаров (50 новых) для Даниелян
✅ Получено 150 товаров (50 новых) для Оганесян
✅ Получено 150 товаров (50 новых) для Старт


In [ ]:
# Достаем данные о каждом товаре
for i in range(len(result)):
    for k in range(len(result[i])):
        product = result[i][k]
        print(product)

In [ ]:
# Объединяем данные обо всех товарах в один список
all_products = [p for acc_data in result if acc_data for p in acc_data]

# Список для хранения словарей с актуальными данными
rows = []

for product in all_products:
    # Информация о товаре
    prod_info = product.get("product", {})
    # Информация о всей статистике
    stat = product.get("statistic", {})
    # Информация о статистике в выбранном периоде
    selected = stat.get("selected", {})

    # Собираем словарь из базовой информации
    row = {
        "account": product.get("account"),
        "nm_id": prod_info.get("nmId"),
        "vendor_сode": prod_info.get("vendorCode"),
        "title": prod_info.get("title"),
        "subject_name": prod_info.get("subjectName"),
        "brand_name": prod_info.get("brandName"),
        "product_rating": prod_info.get("productRating"),
        "feedback_rating": prod_info.get("feedbackRating"),
        "stocks_wb": prod_info.get("stocks", {}).get("wb"),
        "stocks_mp": prod_info.get("stocks", {}).get("mp"),
        "stocks_balance_sum": prod_info.get("stocks", {}).get("balanceSum"),
    }

    # Метрики за выбранный период (selected)
    row.update({
        "open_count": selected.get("openCount"),
        "cart_count": selected.get("cartCount"),
        "order_count": selected.get("orderCount"),
        "orders_sum": selected.get("orderSum"),
        "buyout_count": selected.get("buyoutCount"),
        "buyout_sum": selected.get("buyoutSum"),
        "cancel_count": selected.get("cancelCount"),
        "cancel_sum": selected.get("cancelSum"),
        "avg_price": selected.get("avgPrice"),
        "share_order_percent": selected.get("shareOrderPercent"),
        "add_to_wish_list": selected.get("addToWishlist"),
        "localization_percent": selected.get("localizationPercent"),
    })

    rows.append(row)

In [ ]:
# Создаём DataFrame
df = pd.DataFrame(rows)

# Пример отображения
df.head()